In [63]:
import pandas as pd
import plotly.express as px


from etl_sql.sqlAlchemy.connect import AlchemyConn


engine = AlchemyConn()

df_hotels = pd.read_sql('SELECT hotels.id, hotels.id_ville, hotels.ville, hotels.nom_hotel, hotels.description, hotels.note_hotel, hotels.lat AS lat_hotel ,hotels.lon AS lon_hotel, wheather.description_meteo, wheather.lat, wheather.lon, wheather.precipitation, wheather.temperature FROM hotels INNER JOIN wheather ON hotels.id_ville = wheather.id_ville', con=engine.engine)
df_hotels.set_index('id',inplace=True)
df_hotels.head()

,id_ville,ville,nom_hotel,description,note_hotel,lat_hotel,lon_hotel,description_meteo,lat,lon,precipitation,temperature
id,,,,,,,,,,,,
1,0,Mont Saint-Michel,Mercure Mont Saint Michel,"""Installé dans des espaces verts à seulement 2...",8.3,48.6142,-1.51055,nuageux,48.636,-1.51146,0.0,14.95
2,0,Mont Saint-Michel,Le Duguesclin,"""Situé au Mont-Saint-Michel",8.1,48.6361,-1.51146,nuageux,48.636,-1.51146,0.0,14.95
3,0,Mont Saint-Michel,Le Relais Saint Michel,Le Relais Saint Michel vous accueille face à l...,8.2,48.6176,-1.51040,nuageux,48.636,-1.51146,0.0,14.95
4,0,Mont Saint-Michel,La Vieille Auberge,"""La Vieille Auberge vous accueille dans le vil...",7.5,48.6361,-1.51146,nuageux,48.636,-1.51146,0.0,14.95
5,0,Mont Saint-Michel,La Mère Poulard,"""Séjournez dans un hôtel historique au cœur du...",7.6,48.6351,-1.51054,nuageux,48.636,-1.51146,0.0,14.95


In [64]:
df_hotels.shape

(869, 12)

In [65]:
top5_destinations = df_hotels[['ville','id_ville','description_meteo','precipitation','temperature','lat','lon']].groupby('ville',as_index=False).first()
top5_destinations = top5_destinations.sort_values(by=['temperature', 'precipitation'], ascending=[False, True])
top5_destinations = top5_destinations[:5]
top5_destinations


,ville,id_ville,description_meteo,precipitation,temperature,lat,lon
14,Collioure,27,ciel dégagé,0.0,25.45,42.5251,3.08316
1,Aix-en-Provence,21,ciel dégagé,0.0,25.37,43.5298,5.44747
24,Marseille,20,ciel dégagé,0.0,25.07,43.2962,5.36995
27,Nîmes,24,ciel dégagé,0.0,24.94,43.8374,4.36007
12,Cassis,19,ciel dégagé,0.0,24.61,43.2140,5.53963


In [66]:
plot_temperature = px.scatter_map(top5_destinations,
                                lat='lat',
                                lon='lon',
                                size='temperature',
                                # center={"lat": 48.8566, "lon": 2.3522},
                                hover_name="ville",
                                zoom=4,
                                height=500,
                                map_style="carto-positron",
                                title="Top 5 Destinations (par nombre température)",
                                color='temperature'
                                  )
plot_temperature.show()

In [67]:
top_ville = top5_destinations['ville']

for ville in top_ville : 
    hotels = df_hotels[df_hotels['ville'] == ville]
    top_20_hotels = hotels.sort_values('note_hotel',ascending=False).head(20)
    plot_hotels = px.scatter_map(top_20_hotels,
                                lat='lat_hotel',
                                lon='lon_hotel',
                                size='temperature',
                                center={"lat": top_20_hotels['lat'].mean(), "lon": top_20_hotels['lon'].mean()},
                                hover_name="nom_hotel",
                                zoom=11,
                                height=500,
                                map_style="carto-positron",
                                title=f"Top 20 des hotels à {ville}",
                                color='note_hotel'
                                  )
    plot_hotels.show()
